Return to Live Ocean Files and see if we can do better

In [2]:
import cmocean.cm as cm
import datetime
import gsw
import importlib
import matplotlib.pyplot as plt
import mpl_toolkits.basemap as Basemap
import netCDF4 as nc
import numpy as np
import os
import scipy.interpolate as sinter
import xarray as xr

from salishsea_tools import LiveOcean_grid as grid
from salishsea_tools import LiveOcean_BCs as nancy

%matplotlib inline

In [47]:
importlib.reload(nancy)

<module 'salishsea_tools.LiveOcean_BCs' from '/ocean/sallen/allen/research/Meopar/Tools/SalishSeaTools/salishsea_tools/LiveOcean_BCs.py'>

# Define some stuff

In [3]:
# Create metadeta for temperature and salinity (Live Ocean variables, NEMO grid)
var_meta = {'vosaline': {'grid': 'SalishSea2',
                             'long_name': 'Practical Salinity',
                             'units': 'psu'},
                'votemper': {'grid': 'SalishSea2',
                             'long_name': 'Potential Temperature',
                             'units': 'deg C'},
                'NO3':      {'grid': 'SalishSea2',
                             'long_name': 'Nitrate',
                             'units': 'muM'},
                'Si':       {'grid': 'SalishSea2',
                             'long_name': 'Nitrate',
                             'units': 'muM'},
                'OXY':      {'grid': 'SalishSea2',
                             'long_name': 'Oxygen',
                             'units': 'muM'},
                'DIC':      {'grid': 'SalishSea2',
                             'long_name': 'Dissolved Inorganic Carbon',
                             'units': 'muM'},
                'TA':       {'grid': 'SalishSea2',
                             'long_name': 'Total Alkalinity',
                             'units': 'muM'},
                }

# Mapping from LiveOcean TS names to NEMO TS names
LO_to_NEMO_var_map = {'salt': 'vosaline',
                          'temp': 'votemper',
                          'NO3': 'NO3',
                          'Si': 'Si',
                      'oxygen': 'OXY',
                         'TIC': 'DIC',
                  'alkalinity': 'TA',}


# Load BC information and Specify the New Grid
 

In [4]:
depBC, lonBC, latBC, shape = nancy.load_SalishSea_boundary_grid(imin=376-1, imax=470, rim=10, 
                                                          meshfilename='/home/sallen/MEOPAR/grid/mesh_mask201702.nc')

# Get the Live Ocean File

In [5]:
date = '2018-11-09'
d = nancy.load_LiveOcean(date)

# Process Live Ocean File

## Depth Interpolation

In [22]:
print(d.salt[0].shape)
print(depBC.shape)
print(d.z_rho.values[0, :, 1, 1].shape)
print(d['salt'][0, 0].shape[0])

(30, 98, 57)
(40,)
(30,)
98


In [24]:
interps = nancy.interpolate_to_NEMO_depths(d, depBC, var_names=(var for var in LO_to_NEMO_var_map if var != 'Si'))

## Convert to TEOS-10

In [25]:
new_meta = var_meta.copy()
new_meta['vosaline']['long_name'] = 'Reference Salinity'
new_meta['vosaline']['units'] = 'g/kg'
new_meta['votemper']['long_name'] = 'Conservative Temperature'

In [26]:
sal_ref = gsw.SR_from_SP(interps['salt'])

In [27]:
temp_cons = gsw.CT_from_pt(sal_ref[:], interps['temp'])

In [28]:
interps['salt'] = sal_ref
interps['temp'] = temp_cons
var_meta = new_meta

## Remove South of Tatoosh

In [29]:
interps = nancy.remove_south_of_Tatoosh(interps)

## Fill

In [30]:
interps = nancy.fill_box(interps)

## Convect

In [31]:
sigma = gsw.sigma0(interps['salt'][:], interps['temp'][:] )

In [32]:
sigma, interps = nancy.convect(sigma, interps)

## Extend to Depth

In [33]:
interps = nancy.extend_to_depth(interps)

## Interpolate Horizontally onto NEMO grid

In [34]:
interpl = nancy.interpolate_to_NEMO_lateral(interps, d, lonBC, latBC, shape)

## Convect Again and Stabilize

In [35]:
sigmal = gsw.sigma0(interpl['salt'][:], interpl['temp'][:])

In [36]:
print(np.nanmax(sigmal[:-1, :, :] - sigmal[1:, :, :]))
sigmal, interpl = nancy.convect(sigmal, interpl)
interpl = nancy.stabilize(sigmal, interpl)
print(np.nanmax(sigmal[:-1, :, :] - sigmal[1:, :, :]))

0.0
0.0


## Reorder Array

In [37]:
for var in interpl.keys():
    interpl[var] = np.swapaxes(interpl[var], 1, 2)
    interpl[var] = interpl[var].reshape(1, interpl[var].shape[0], 1, interpl[var].shape[2]*interpl[var].shape[1])


## Calculate Quantities LiveOcean doesn't Track like SI

In [38]:
interpl['Si'] = nancy.calculate_Si_from_NO3(interpl['NO3'])

# Write Out File

In [39]:
ts = d.ocean_time.data
ds = nancy.prepare_dataset(interpl, var_meta, LO_to_NEMO_var_map, depBC, ts)

In [40]:
ds

<xarray.Dataset>
Dimensions:       (deptht: 40, time_counter: 1, xbT: 950, yb: 1)
Coordinates:
  * time_counter  (time_counter) datetime64[ns] 2018-11-10T12:00:00
  * yb            (yb) int64 1
  * deptht        (deptht) float64 0.5 1.5 2.5 3.5 4.5 5.5 6.5 7.501 8.501 ...
  * xbT           (xbT) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 ...
Data variables:
    vosaline      (time_counter, deptht, yb, xbT) float64 30.46 30.46 30.46 ...
    TA            (time_counter, deptht, yb, xbT) float64 2.174e+03 ...
    NO3           (time_counter, deptht, yb, xbT) float64 8.086 8.086 8.086 ...
    DIC           (time_counter, deptht, yb, xbT) float64 1.985e+03 ...
    OXY           (time_counter, deptht, yb, xbT) float64 276.2 276.2 276.2 ...
    Si            (time_counter, deptht, yb, xbT) float64 17.38 17.38 17.38 ...
    votemper      (time_counter, deptht, yb, xbT) float64 10.65 10.65 10.65 ...
Attributes:
    creator_name:          Salish Sea MEOPAR Project Contributors
    summ

In [41]:
basename = 'single_LO'
sdt = datetime.datetime.strptime(date, '%Y-%m-%d')
filename = '{}_{}.nc'.format(
                basename, sdt.strftime('y%Ym%md%d'))
print(filename)

single_LO_y2018m11d09.nc


In [42]:
encoding = {
    var: {
       'zlib': True
    }
    for var in ds.data_vars
}
encoding['time_counter'] = {'units': 'minutes since 1970-01-01 00:00'}

In [43]:
ds.to_netcdf(path='stuffit.nc', unlimited_dims=('time_counter'),
             encoding=encoding, 
             )       

In [44]:
os.rename('stuffit.nc', filename)

In [45]:
ds.close()

# Just use Module

In [48]:
importlib.reload(nancy)

<module 'salishsea_tools.LiveOcean_BCs' from '/ocean/sallen/allen/research/Meopar/Tools/SalishSeaTools/salishsea_tools/LiveOcean_BCs.py'>

In [49]:
filepath = nancy.create_LiveOcean_TS_BCs('2018-11-09', bc_dir='./')

In [ ]:
before = nc.Dataset('/results/forcing/LiveOcean/boundary_conditions/LiveOcean_v201712_y2018m02d19.nc')
after = nc.Dataset('./LiveOcean_v201712_y2018m02d19.nc')
before = nc.Dataset('./LiveOcean_v201712_y2018m02d19.nc.nosalt')

In [ ]:
salt_before = before.variables['vosaline'][0]
salt_after = after.variables['vosaline'][0]
salt_before.shape

In [ ]:
fig, ax = plt.subplots(1,3 ,figsize=(20,5))
mesh = ax[0].pcolormesh(salt_before[:,0], vmax=34.1, vmin = 31.5)
fig.colorbar(mesh, ax=ax[0])
mesh = ax[1].pcolormesh(salt_after[:,0], vmax=34.1, vmin = 31.5)
fig.colorbar(mesh, ax=ax[1])
mesh = ax[2].pcolormesh(salt_after[:, 0] - salt_before[:, 0], cmap=cm.curl, vmax=0.1, vmin=-0.1)
fig.colorbar(mesh, ax=ax[2])
for ax in [ax[0], ax[1], ax[2]]:
    ax.invert_yaxis()
print (np.max(salt_after - salt_before), np.min(salt_after - salt_before))

In [ ]:
fig, ax = plt.subplots(1,3 ,figsize=(20,5))
mesh = ax[0].pcolormesh(salt_before[1:, 0] -salt_before[:-1, 0], cmap='bwr', vmax=0.1, vmin=-0.1)
fig.colorbar(mesh, ax=ax[0])
mesh = ax[1].pcolormesh(salt_after[1:, 0] - salt_after[:-1, 0], cmap='bwr', vmax=0.1, vmin=-0.1)
fig.colorbar(mesh, ax=ax[1])
mesh = ax[2].pcolormesh(salt_after[:, 0] - salt_before[:, 0], cmap='bwr', vmax=0.1, vmin=-0.1)
fig.colorbar(mesh, ax=ax[2])
for ax in [ax[0], ax[1], ax[2]]:
    ax.invert_yaxis()

In [ ]:
temp_before = before.variables['votemper'][0]
temp_after = after.variables['votemper'][0]

In [ ]:
fig, ax = plt.subplots(1,3 ,figsize=(20,5))
mesh = ax[0].pcolormesh(temp_before[1:, 0] - temp_before[0:-1, 0], cmap='bwr', vmin=-0.6, vmax=0.6 )
fig.colorbar(mesh, ax=ax[0])
mesh = ax[1].pcolormesh(temp_after[1:, 0] -temp_after[0:-1, 0], cmap='bwr', vmin=-0.6, vmax=0.6)
fig.colorbar(mesh, ax=ax[1])
mesh = ax[2].pcolormesh(temp_after[:, 0] - temp_before[:, 0], cmap='bwr', vmax=0.1, vmin=-0.1)
fig.colorbar(mesh, ax=ax[2])
for ax in [ax[0], ax[1], ax[2]]:
    ax.invert_yaxis()